<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/tpu/fine_tune_tpu_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Xception TPUs (Colab)&nbsp;<img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50">

### Enabling and testing the TPU

In [1]:
import tensorflow as tf

tf.__version__

'2.3.0'

In [13]:
#@title Enagle TPU
use_tpu = True #@param ["False", "True"] {type:"raw"}

if use_tpu:
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tpu_spec = tpu.cluster_spec().as_dict()['worker']
    print('Running on TPU:', tpu_spec)
  except ValueError:
    print('ERROR: Not connected to a TPU.')

ERROR: Not connected to a TPU.


In [3]:
print(use_tpu)

True
